In [27]:
from aerolib import *
import pandas as pd

In [22]:
def isNaN(num):
    return num != num

In [23]:
class InsufficientInputData(Exception):
    def __init__(self, text):
        self.txt = text

In [6]:
class Foil():
    def __init__(self, name):
        pass
    def get_geometry(self):
        pass
    def get_filename(self):
        pass
    

In [1]:
class Sculptor():
    def __init__(self, performance_file_name, params_file_name, settings_file_name, foils_file_names, m):
        df = pd.read_csv(performance_file_name, header=None)
        self.performance = dict(zip(df[0], df[1]))
        self.preprocess_performance()

        df = pd.read_csv(settings_file_name, header=None)
        self.settings = dict(zip(df[0], df[1]))

        self.params=pd.read_csv(params_file_name, index_col=False).T
        
        self.foils = []
        for name in foils_file_names:
            self.foils(Foil(name))

        self.geom = {}
        self.aero = {"cruise":np.nan}
        self.tom = m
        self.preprocess_input_data()

    #FIXME: не дописаны critical_params
    #(обозначения тут правильные)
    def preprocess_input_data(self):
        if not self.settings['g']:
            self.settings['g'] = 9.81
            
        if not self.performance['wire_scale_coef']:
            self.performance['wire_scale_coef'] = 1.3
            
        if not self.settings['Density']:
            self.settings['Density'] = 1.22
        
        critical_params = ['TOM', 'take_off_speed', 'CL_take_off', 
                           'eta_prop', 'cruise_speed', 'A_aft', 
                           'B_keel', 'l_stab', 'wire_scale_coef']
        for param in critical_params:
            if isNaN(self.performance.get(param, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')

    #FIXME
    #подставить аргументы в функции из aerolib
    def calculate_geometry(self):        
        self.geom["wing_area"] = wing_area(self.tom, float(settings["g"]), 
                                              float(settings["Density"]), 
                                               float(performance["take_off_speed"]))
        self.CL_cr = CL_cruise(self.tom, self.performance["cruise_speed"],
                              float(self.geom["wing_area"]), float(self.settings["g"]),
                              float(self.settings["Density"]))
        ar_step_number = (float(self.settings["ar_min"]) - float(self.settings["ar_max"])) // float(self.settings["ar_delta"])
        ar_range = np.linspace(float(self.settings["ar_min"]), float(self.settings["ar_max"]), ar_step_number)
        self.geom["AR"] = AR_selector(ar_range, self.geom, self.settings,
                                                        self.performance, self.tom)
        self.aero["cruise"] = K_V_solver(self.geom, self.settings, self.performance["cruise_speed"], 
                                         self.geom["AR"], self.tom)
        self.geom["ba"] = ba(self.geom["AR"], self.geom["wing_area"])
        self.geom["wingspan"] = wingspan(self.geom["AR"], self.geom["wing_area"])
        self.geom["aft_area"] = aft_area(float(self.params["A_aft"]), self.geom["wing_area"],
                                        self.geom["ba"], float(self.params["l_stab"]))
        self.geom["keel_area"] = keel_area(float(self.params["B_keel"]), self.geom["wing_area"],
                                          self.geom["wingspan"], float(self.params["l_stab"]))
        self.geom["V_dihedral"] = gamma(self.geom["keel_area"], self.geom["aft_area"])
        self.geom["Vtail_area"] = stab_area(self.geom["aft_area"], self.geom["V_dihedral"])
        self.geom["Pcruise"] = P_cruise(self.tom, self.aero["cruise"].K, 
                                        float(self.params["eta_prop"]), float(self.settings["g"]))
        self.geom["wirelength"] = wire_length(self.geom["wingspan"], float(self.params["l_stab"]),
                                             float(self.settings["wire_self_coef"]))
        return 0
        
    def update_m(self, new_m):
        self.tom = new_m
    
    def write_geometry(self):
        with open("geometry.csv", 'w') as f:
            for key in self.geomatry.keys():
                f.write("%s, %s\n" % (key, self.geomatry[key]))

    def get_data_to_weigh():
        return [self.params, self.geometry]

In [37]:
'''  
Ожидаемый формат params-csv-файла
NAME  INDEX  VALUE CONJUGATE
0   m1    1.0      1            NaN
1   n1    1.0     10            NaN
2   m2    2.0      2            NaN
3   n2    2.0    100            NaN
4   m3    NaN      3            NaN
5    m    NaN   1000             cy

Ожидаемый формат geomerty-csv-файла
NAME      VALUE
wing_area 1
ba        1
wingspan  1
AR        1
'''

#m (масса ЛА) -- есть сумма
#слагаемые бывают трёх типов:
#1) произведение величин из params с одинаковым индексом
#2) произведение двух величин -- одна из params, одна из geometry (её имя - CONJUGATE)
#3) величина из params
def weigh(params_file_name, geometry_file_name):
    geom_df = pd.read_csv(geometry_file_name, header=None).loc[1:] #удаление заголовка из DF
    geom = dict(zip(geom_df[0], geom_df[1])) #создание словаря {Cy: <val>, ...}

    #создание таблицы с колонками вида 
    #[<colomn_num>, <NAME>, <INDEX>, <VALUE>, <CONJUGATE>]
    params=pd.read_csv(params_file_name, index_col=False).T 

    #создание словаря 
    #{<значение колонки index из params>: <список значений всех параметров с этим индексом>}
    multiply_dict = dict()
    m = 0
    
    for var in params: #итерация по колонкам; в var попадает номер колонки (colomn_num)
        if isNaN(params[var]["VALUE"]):
            raise InsufficientInputData(f"no value for variable '{params[var]['NAME']}'")

        if not isNaN(params[var]["INDEX"]): #если тип (1)
            #создать в multiply_dict ключ с найденым индексом,
            #со значением пустого списка, если такого ключа ещё нет
            multiply_dict[params[var]["INDEX"]] = multiply_dict.get(params[var]["INDEX"], [])
            #добавить значение этого параметра
            #в список индекса найденного индекса
            multiply_dict[params[var]["INDEX"]].append(params[var]["VALUE"])
        
        elif not isNaN(params[var]["CONJUGATE"]): #если тип (2)
            if not isNaN(geom.get(params[var]["CONJUGATE"], np.nan)):  
                m += geom[params[var]["CONJUGATE"]] * params[var]["VALUE"]
            else:
                raise InsufficientInputData(f"for variable '{params[var]['NAME']}', '{params[var]['CONJUGATE']}' "+\
                                            f"is specified as conjugate, but it is no '{params[var]['CONJUGATE']}'"+\
                                            f" in the performance-file")
                
        elif not isNaN(params[var]["VALUE"]): #если тип (3)
            m += params[var]["VALUE"]

    #вычисление слагаемых типа (1) -- перемножение значения параметров с одинаковыми индексами
    for vals in multiply_dict.values():
        m += math.prod(vals)
    return m


In [38]:
weigh("PARAMS.csv", "GEOM.csv")

3.45

In [ ]:
# Модуль расчёта геометрических характеристик потребляет на вход: 
# dataframe PERFORMANCE с потребными эксплуатационными характеристиками
# dataframe PARAMS с параметрами электронных компонент, конструкционных материалов, аккумуляторов и т.д.
# пути до файлов селиговского формата с профилем крыла wing_foil и с профилем оперения aft_foil.
# TOM - значение взлётной массы в начальном приближении.

# PERFORMANCE включает параметры take_off_speed, cruise_speed, flight_time, payload

# PARAMS включает параметры m_FPV, m_powerplant, m_flight_control, m_fus, m_servo1, m_servo2,
# line_dens_wire, line_dens_tube1, line_dens_tube2, line_dens_tube3,
# area_dens_LWPLA
# energy_dens_bat
# number_servo1, number_servo2
# l_stab

# Все единицы в СИ
def inner_iteration(PERFORMANCE, PARAMS, wing_foil, aft_foil, TOM):
    performance_file_name = input("имя файла performances: ")
    params_file_name = input("имя файла params: ")
    settings_file_name = input("имя файла settings: ") 
    foils_file_names = input("имена файлов с профилями (через запятую): ")
    tom = input("нулевое приближение влётной массы: ")
    max_iter = input("максимальное число итераций: ")
    freq_geom_saving = input("частота сохранения промежуточных геометрии: ")
    
    df = pd.read_csv(settings_file_name, header=None)
    tom_eps = dict(zip(df[0], df[1]))["tom_eps"]
    
    sculptor = Sculptor(performance_file_name, params_file_name, settings_file_name, foils_file_names, m)
    i = 0
    while True:
        sculptor.calculate_geometry()
        new_tom = weigh(sculptor.get_data_to_weigh())
        if abs(tom - new_tom) > tom_eps:
            tom = new_tom
            sculptor.update_m(new_tom)
        else:
            print(f"сошлось на итерации: {i}")
            sculptor.write_geometry()
            print("информация сохранена в файл с геометрией")
            break
        
        if i == max_iter:
            print(f"прошло {i} итераций, но расчёт всё ещё не завершён.")
            flag = 2
            while flag not in ['0', '1']:
                flag = input(f"введите 1, чтобы произвести ещё {max_iter} операций, иначе 0: ")
            if flag:
                i = 0
            else:
                sculptor.write_geometry()
                print("последняя итерация геометрии сохранена")
                break
                
        if i%freq_geom_saving == 0:
            sculptor.write_geometry()
            
        i+=1
    